# Train model using trace dataset, funtion,Feb 01,2021

# XY no saved, since it take too much storage

In [14]:
import csv
import os
import pandas as pd
import numpy as np
from numpy import argmax
import pickle
import pdb
import functions as f
from functions import convert_to_binary,create_windowed_dataset
import glob
from tqdm import tqdm
from tensorflow.keras import Model, Input
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence
from tcn import TCN
from tensorflow.keras.models import load_model, Sequential
from sklearn.metrics import roc_curve,f1_score,recall_score,precision_score,accuracy_score
import pandas as pd
from tcn_model import tcn_model
pd.options.mode.chained_assignment = None 


## Functions

### Function: generate X and Y

In [15]:
#data_file=file_list[1]

def gen_XY(data_file,file_name,Length=None):
    from functions import convert_to_binary,create_windowed_dataset
    if Length!=None:
        df = pd.read_pickle(data_file)[0:Length]
    else:
        df = pd.read_pickle(data_file)
    #%% Binarize 

    BINARY_BIT_SIZE = 64-6
    look_back = 3

    #b_list = ["delta_token", "page_address", "page_offset", "cache_line_index"]
    b_list = ["page_cache_index"]
    for item in b_list:
        X = np.array(df[item]).reshape(-1,1)
        s=item+"_bin"
        df[s]=list(convert_to_binary(data=X,bit_size=BINARY_BIT_SIZE))
    print("Binarization completed")
    #%% X: input data
    X = create_windowed_dataset(list(df["page_cache_index_bin"]), look_back)
    X = X.reshape(len(X),-1)
    print("Look-back window completed")
    BITMAP_SIZE = 64
    def to_bitmap(n): 
        l0=np.zeros((BITMAP_SIZE),dtype = int)
        if(len(n)>0):
            for x in n:
                l0[x]=1
            l1=list(l0)
            l1.reverse()
            return l1
        else:
            return list(l0)
    print("Bitmap completed")   
    numbers=list(df["label"][look_back:])
    result = map(to_bitmap, numbers) 
    #print(list(result)) 

    #df["label_bitmap"]=list(result)

    y=np.array(list(result))
    df_x=df[look_back:]
    df_x['X']=list(X)
    df_x['y']=list(y)

    print(X.shape,y.shape)
    #df_x.to_pickle(XY_SAVE_PATH+file_name+"_xy.pk")
    #print("pickle saved:",XY_SAVE_PATH+file_name+"_xy.pk")
    
    split1=len(df_x)//2
    split2=len(df_x)*3//4
    
    df_train=df_x[0:split1]
    df_val=df_x[split1:split2]
    df_eva=df_x[split2:]
    
    return df_train,df_val,df_eva

### Function: Train_test_split 

In [16]:
def df_to_XY(df_train,df_val):

    X_train = np.stack(df_train['X'])
    y_train =np.stack(df_train['y'])

    X_val = np.stack(df_val['X'])
    y_val = np.stack(df_val['y'])

    print("X shape:",X_train.shape,X_val.shape)
    print("y shape:",y_train.shape,y_val.shape)
    
    return X_train,y_train,X_val,y_val

### Function:Train

In [17]:
#%%% Model config
def train(model_file_name,X_train,y_train,embedding_dim,batch_size,num_epochs,tcn_num_filters):
    '''train model'''
    i_dim=X_train.shape[1]
    o_dim=y_train.shape[1]

    model_ = tcn_model(final_vocab_size, batch_size,embedding_dim, i_dim, o_dim,tcn_num_filters)
    history=model_.train(X_train, y_train,X_val, y_val, num_epochs, batch_size)
    model_.model.save(model_file_name)
    print("model saved")

### Function:Evaluate

In [18]:
def evaluate_k(dataset,model_path, K=None):
    if K!=None:
        dataset['diff']=dataset['page_address'].diff() 
        dataset['1']=[ bool(x) for x in dataset['diff']]# first hit
       # K_HITS=5
        if K>1:
            for i in range(2,K+1):
                col=str(i)
                col_pre=str(i-1)
                dataset['diff']=dataset[col_pre].diff() # second hit: diff2=-1
                dataset[col]= dataset[col_pre] | dataset['diff']
               # print(dataset)

        df_k_hits=dataset.loc[dataset[str(K)]==True]
    else:
        df_k_hits=dataset
    X_test=np.stack(df_k_hits['X'])
    y_test=np.stack(df_k_hits['y'])
    
    model_ = load_model(model_path, custom_objects={'TCN': TCN})
    y_score = model_.predict(X_test)
    
    y_pred_bin=y_score
    y_pred_bin[y_pred_bin >= 0.5] = 1
    y_pred_bin[y_pred_bin < 0.5] = 0
    
    #f1_score:tp / (tp + fp)
    f1_score_res=f1_score(y_test, y_pred_bin, average='samples')
    #recall: tp / (tp + fn)
    recall_score_res=recall_score(y_test, y_pred_bin, average='samples')
    #precision: tp / (tp + fp)
    precision_score_res=precision_score(y_test, y_pred_bin, average='samples',zero_division=0)

    #print("f1_score:",f1_score_res)
    #print("recall_score:",recall_score_res)
    #print("precision_score:",precision_score_res)
    return f1_score_res,recall_score_res,precision_score_res

### Function: Evaluate k

In [19]:
def evaluate_k_hits(df_eva,model_file_name,k_hits):
    f_l=[];r_l=[];p_l=[]
    for i in tqdm(range(1,k_hits+1)):#65
        f,r,p=evaluate_k(df_eva,model_file_name, i)
        f_l.append(f);r_l.append(r);p_l.append(p)
    
    res_dic={'f1 score':f_l,'recall':r_l,'precision':p_l}
    df_res=pd.DataFrame(res_dic,columns=['f1 score','recall','precision'])
    df_res.to_pickle(eva_save_file)

## Configuration

In [20]:
M,N=200,64
LEN=400000
DATA_PATH="./dataset_pickle/dataset1_M_"+str(M)+"_N_"+str(N)+"/"
#"./dataset_pickle/dataset1_M_500_N_64/"
#XY_SAVE_PATH=DATA_PATH+"XY/"
MODEL_SAVE_PATH = DATA_PATH+"model/"
EVA_RESULT_SAVE_PATH = DATA_PATH+"eva/"
BASELINE_SAVE_DIR = DATA_PATH+"baseline/"

file_list = sorted(glob.glob(DATA_PATH+"*.pk"))

if not os.path.exists(MODEL_SAVE_PATH):
    os.makedirs(MODEL_SAVE_PATH)

if not os.path.exists(EVA_RESULT_SAVE_PATH):
    os.makedirs(EVA_RESULT_SAVE_PATH)
    
if not os.path.exists(BASELINE_SAVE_DIR):
    os.makedirs(BASELINE_SAVE_DIR)
    

    
final_vocab_size=2
embedding_dim=20
batch_size=256
num_epochs=20
tcn_num_filters = 64
k_hits=64

## Main

## Only evaluate all

In [21]:
Length=1000
df_res=pd.DataFrame([])
for file in file_list:
    print("Start: ", file)
    app_name=file.split(".pk")[0].split("/")[-1]
    model_file_name = MODEL_SAVE_PATH+app_name+".h5"
    eva_save_file = EVA_RESULT_SAVE_PATH + app_name+"eva.pk"
    
    df_train,df_val,df_eva=gen_XY(file,app_name,Length)
    X_train,y_train,X_val,y_val = df_to_XY(df_train,df_val) 
    train(model_file_name,X_train,y_train,embedding_dim,batch_size,num_epochs,tcn_num_filters)
    f,r,p = evaluate_k(df_eva,model_file_name)
    df_res[app_name]=[f,r,p]
    #evaluate_k_hits(df_eva,model_file_name,k_hits)
    print("Done for: ", file)
    np_save_path=BASELINE_SAVE_DIR+"eva_all.pk"
    df_res.to_pickle(np_save_path)

Start:  ./dataset_pickle/dataset1_M_200_N_64/600.perlbench.pk
Binarization completed
Look-back window completed
Bitmap completed
(997, 232) (997, 64)
X shape: (498, 232) (249, 232)
y shape: (498, 64) (249, 64)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 232, 20)           40        
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 232, 64), (None,  17600     
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 232, 64), (None,  24704     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 232, 64), (None,  24704     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 232, 64), (None,  24704     
__________________________________________

In [22]:
np_save_path=BASELINE_SAVE_DIR+"eva_all.pk"
df_res=pd.read_pickle(np_save_path)
df_res

,600.perlbench
0,0.52
1,0.52
2,0.52
